In [5]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Lesson 8 - Matrix Multiplication

In [2]:
import torch

In [3]:
x = torch.rand((5, 784))
w = torch.rand(784, 10)

In [4]:
x.shape, w.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

In [5]:
%timeit torch.matmul(x, w)

4.11 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [6]:
%timeit x@w

4.06 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
c = torch.zeros((5, 10))

In [8]:
def matmul(x, w):
    for i in range(x.shape[0]):
        for j in range(w.shape[1]):
            for k in range(w.shape[0]):
                c[i,j] += x[i, k]*w[k, j]
    return c

In [14]:
%timeit matmul(x, w)

516 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
509000/4

127250.0

In [15]:
def matmul_elop(x, w):
    for i in range(x.shape[0]):
        for j in range(w.shape[1]):
            c[i,j] = (x[i, :]*w[:, j]).sum()
    return c

In [16]:
%timeit matmul_elop(x, w)

752 µs ± 5.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
509000/740

687.8378378378378

In [38]:
def matmul_elop_bc(x, w):
    for i in range(x.shape[0]):
        c[i,:] = (x[i, :].unsqueeze(1)*w).sum(dim=0)
    return c

In [39]:
%timeit matmul_elop_bc(x, w)

166 µs ± 611 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [41]:
matmul_elop_bc(x, w).shape

torch.Size([5, 10])

In [42]:
752/166

4.530120481927711

In [44]:
%timeit torch.einsum('ij,jk->ik', x, w)

23.5 µs ± 603 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [45]:
166/24

6.916666666666667

In [46]:
%timeit x.matmul(w)

3.94 µs ± 83.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## L8 - Forward

In [9]:
import fastai

In [10]:
fastai.__version__

'1.0.51.dev0'

In [11]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

VersionConflict: (fastprogress 0.1.18 (/mnt/datadrive1/anaconda3/envs/fastaiv3_devi/lib/python3.6/site-packages), Requirement.parse('fastprogress>=0.1.19'))

In [53]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

SyntaxError: invalid syntax (<ipython-input-53-d4d509e3ed3d>, line 2)

In [48]:
x_train,y_train,x_valid,y_valid = get_data()

NameError: name 'datasets' is not defined